# NYC Flights 2013 Analysis
data description is via https://cran.r-project.org/web/packages/nycflights13/nycflights13.pdf.

In [ ]:
# shift + enter ==> RUN CODE
# ctrl + M ==> convert code cell to markdown list

In [ ]:
library(tidyverse)
library(glue)
library(dplyr)

In [ ]:
flights <- read_csv("flights.csv")
airlines <- read_csv("airlines.csv")
airports <- read_csv("airports.csv")

Rows: 336776 Columns: 19

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (4): carrier, tailnum, origin, dest
dbl  (14): year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, ...
dttm  (1): time_hour


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 16 Columns: 2

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): carrier, name


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 1458 Columns: 8

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): faa, name, dst, tzone
dbl (4): lat, lon, alt, tz


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or 

In [ ]:
glimpse(flights)

Rows: 336,776
Columns: 19
$ year           <dbl> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <dbl> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <dbl> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <dbl> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <dbl> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <dbl> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

In [ ]:

glimpse(airlines)

Rows: 16
Columns: 2
$ carrier <chr> "9E", "AA", "AS", "B6", "DL", "EV", "F9", "FL", "HA", "MQ", "O…
$ name    <chr> "Endeavor Air Inc.", "American Airlines Inc.", "Alaska Airline…


In [ ]:

glimpse(airports)

Rows: 1,458
Columns: 8
$ faa   <chr> "04G", "06A", "06C", "06N", "09J", "0A9", "0G6", "0G7", "0P2", "…
$ name  <chr> "Lansdowne Airport", "Moton Field Municipal Airport", "Schaumbur…
$ lat   <dbl> 41.13047, 32.46057, 41.98934, 41.43191, 31.07447, 36.37122, 41.4…
$ lon   <dbl> -80.61958, -85.68003, -88.10124, -74.39156, -81.42778, -82.17342…
$ alt   <dbl> 1044, 264, 801, 523, 11, 1593, 730, 492, 1000, 108, 409, 875, 10…
$ tz    <dbl> -5, -6, -6, -5, -5, -5, -5, -5, -5, -8, -5, -6, -5, -5, -5, -5, …
$ dst   <chr> "A", "A", "A", "A", "A", "A", "A", "A", "U", "A", "A", "U", "A",…
$ tzone <chr> "America/New_York", "America/Chicago", "America/Chicago", "Ameri…


# Q01: Which airline had the most flights on Christmas?
Answer: JetBlue Airways

In [ ]:
flights %>%
filter(month == 12, day == 25) %>% #filter specific date.
count(carrier) %>% # count n_total of each airline.
arrange(desc(n)) %>% # order desc to sort largest to smallest.
left_join(airlines, "carrier") %>% # join airlines table to get a full name of each airline.
select(carrier_name = name, n_flights = n) %>% 
head(1) # pull airline with the highest n_total.

carrier_name,n_flights
<chr>,<int>
JetBlue Airways,159


# Q02: Where were the top 3 destinations in which JetBlue Airways arrived on Christmas? 
Answer: (1) Fort Lauderdale Hollywood Intl
(2) Orlando Intl
(3) Palm Beach Intl

In [ ]:
flights %>%
filter(month == 12, day == 25, carrier == "B6") %>% # filter specific date and the focused airline.
count(dest) %>% # count n_total of each destination.
arrange(desc(n)) %>% # order desc to sort largest to smallest.
left_join(airports, by = c("dest" = "faa")) %>% # join with abbreviation of each destination to get full name.
select(destination = name, n)%>%
head(3) # pull the first 3 destination having n_total in order.

destination,n
<chr>,<int>
Fort Lauderdale Hollywood Intl,23
Orlando Intl,20
Palm Beach Intl,14


# Q03: Which airline had the most departure flights delayed in 2013?
Answer: United Air Lines Inc.

In [ ]:
airlines %>%
mutate(airlines_name = name) %>% # create a new column whose column name is more specific.
select(carrier, airlines_name) %>% 
left_join(flights, "carrier") %>% # join with carrier abbreviation to get more details from flights table.
select(carrier, airlines_name, dep_delay) %>%
filter(dep_delay > 0, !is.na(dep_delay)) %>% # filter negative number or blank out to get data of flights being delayed only. 
count(airlines_name) %>% 
arrange(desc(n)) %>% # order desc to sort largest to smallest.
group_by(airlines_name) %>% # group by each airline to see how many times each flight delayed.
head(1)

airlines_name,n
<chr>,<int>
United Air Lines Inc.,27261


# Q04: What was the monthly average departure flight delayed by United Air Lines?

In [ ]:
flights %>%
filter(carrier == "UA") %>% # filter specific airline
select(carrier, month, dep_delay) %>%
filter(dep_delay > 0, !is.na(dep_delay)) %>% # filter negative number or blank out to get data of flights being delayed only. 
group_by(month) %>%
summarise(avg_dep_delay = mean(dep_delay))

month,avg_dep_delay
<dbl>,<dbl>
1,22.86570
2,22.83492
3,29.79762
4,35.25358
5,28.73297
6,38.89502
7,38.54497
8,28.86060
9,29.12829


# Q05: What was the busiest day in December?
Answer: Monday

In [ ]:
flights %>%
mutate(full_date = substr(time_hour, 1, 10)) %>% # create a new column as a full date.
mutate(day = strftime(full_date, "%A")) %>% # convert full date to be day of week.
filter(month == "12") %>% # filter month to be specific.
select(day, month, flight) %>%
count(day) %>% # count n_total of each day of week
arrange(desc(n)) %>% # order desc to sort largest to smallest
group_by(day) # group by day of week.

day,n
<chr>,<int>
Monday,4888
Tuesday,4558
Sunday,4138
Friday,3871
Thursday,3777
Wednesday,3661
Saturday,3242
